In [249]:
import numpy as np
import time
from scipy import linalg

# Maxvol

In [250]:
def Maxvol(A, eps = 1.06, iter=10):
    n, r = A.shape
    for _ in range(iter):
        i = A.shape[1]
        C = A[:i, :]
        B = A @ np.linalg.inv(C)
        index = np.unravel_index(np.argmax(B), B.shape)
        if np.round(np.abs(B[index]), 10) == 1:
            return C
        if np.round(np.abs(B[index]), 10) > 1:
            A[index, :] = A[[index[1], index[0]], :]
    return C

In [251]:
r = 3
n = 10
A = np.zeros([n, r])
for i in range(n):
    for j in range(r):
        A[i,j]= np.cos(2*i - j)
Maxvol(A)

array([[ 1.        ,  0.54030231, -0.41614684],
       [-0.41614684,  0.54030231,  1.        ],
       [-0.65364362, -0.9899925 , -0.41614684]])

# Cross

In [252]:
def Cross(A, eps = 0.1):
    r = 0
    m, n = A.shape
    I = np.arange(m)
    J = np.arange(n)
    i,j = np.unravel_index(np.argmax(np.abs(A), axis=None), A.shape)
    V = np.copy(A[:, j] / A[i, j]).reshape(m, 1)
    U = np.copy(A[i, :]).reshape(1, n)
    Error = A - V @ U
    I[i] = -1
    J[j] = -1
    i = 0
    j = 0
    while True:
        r += 1
        for k in range(n):
            if (abs(Error[k,j]) > abs(Error[i,j])) and (I[k] != -1):
                i = k
            if (abs(Error[i,k]) > abs(Error[i,j])) and (J[k] != -1):
                j = k
        if ((abs(Error[i][j]) *((m-r)*(n-r))**0.5) <= eps*(np.linalg.norm(V @ U, ord='fro'))):
            r -= 1
            return V, U
        U = np.concatenate((U, Error[i, :].reshape(1, n)), axis = 0)
        V = np.concatenate((V, (Error[:, j] / Error[i,j]).reshape(m, 1)), axis = 1)
        I[i] = -1
        J[j] = -1
        Error = A - V @ U

In [253]:
n = 16
m = 64
A = np.zeros([m, n])
for i in range(m):
    for j in range(n):
        A[i,j] = i*2 - j

In [254]:
U, V = Cross(A, 0.01)
np.linalg.norm((A - U @ V), ord = 'fro')

1.3916028932457436e-13

# Randomized SVD

In [255]:
def Randomized_SVD(A, r, p = 8):
    M, N = A.shape
    B = A @ (np.random.normal(size = (N, r + p)))
    Q, R = np.linalg.qr(B)
    U, E, V = np.linalg.svd(Q.T @ A);
    return (Q @ U, E, V)

def A(i, j):
    return i * j

M = 1024
N = 2048
r = 4
U, E, V = Randomized_SVD(np.fromfunction(A, (M, N)), r)

In [256]:
U

array([[-5.35758610e-17,  2.65210406e-02,  3.33518372e-01, ...,
        -1.06062371e-01, -7.22966898e-03,  1.49261245e-02],
       [-5.28967401e-05, -1.46422625e-04,  7.83868676e-03, ...,
        -1.08497403e-03,  2.19474951e-04,  9.66930216e-03],
       [-1.05793480e-04,  1.18725451e-03, -1.17975440e-02, ...,
         3.83698576e-03, -1.11026098e-03, -1.87729177e-02],
       ...,
       [-5.40075717e-02, -1.70839429e-02,  7.08684542e-06, ...,
         2.20890088e-02,  5.10796364e-02,  1.99716950e-02],
       [-5.40604684e-02, -2.89313035e-02, -2.68058115e-03, ...,
        -3.57440562e-04, -2.92910666e-02,  1.12962749e-01],
       [-5.41133652e-02,  6.41564003e-02, -1.32842188e-02, ...,
         5.25850050e-03,  3.11398690e-02, -2.57123341e-02]])

In [257]:
E

array([1.01122155e+09, 2.33042743e-08, 2.03200538e-08, 1.89310678e-08,
       1.75277070e-08, 1.61425641e-08, 1.39083732e-08, 1.25732269e-08,
       1.15490620e-08, 1.12332866e-08, 1.06585150e-08, 8.97748799e-09])

In [258]:
V

array([[ 0.00000000e+00, -1.86949701e-05, -3.73899402e-05, ...,
        -3.82312138e-02, -3.82499088e-02, -3.82686038e-02],
       [-1.30475072e-16,  7.46522920e-06,  1.49304584e-05, ...,
        -3.83187267e-02,  3.68045250e-02, -3.58633038e-02],
       [-2.15446221e-15,  9.34585899e-06,  1.86917180e-05, ...,
        -4.02383762e-02,  6.25364513e-02, -2.42422418e-02],
       ...,
       [-3.82302517e-02, -8.96579681e-04,  3.97878316e-02, ...,
         9.91535829e-01,  4.02525297e-03, -7.41109627e-04],
       [-3.82504270e-02,  1.03816988e-01, -4.20154185e-02, ...,
         4.02644263e-03,  9.72114020e-01,  1.25007438e-03],
       [-3.82566852e-02,  4.23354667e-02,  3.47161503e-02, ...,
        -7.45487195e-04,  1.25692764e-03,  9.65896644e-01]])

# HO_SVD

### 3-мерный HO_SVD

In [259]:
def HO_SVD_3d(A, r1, r2, r3):
    n1,n2,n3 = A.shape
    A1 = A.reshape(n1, n2*n3)
    U, s, V = np.linalg.svd(A1, full_matrices=False)
    V1 = U[:,:r1] 
    V1 = V1.T
    A1 = np.diag(s[:r1]) @ V[:r1,:] # r1 x n2*n3
    A1 = A1.reshape(r1, n2, n3)
    A1 = A1.transpose([0,2,1])
    A1 = A1.reshape(r1 * n3, n2)# r1 * n3 x n2
    U, s, V = np.linalg.svd(A1, full_matrices=False)
    V2 =  V[:r2,:] # r2 x n2
    U2 = U[:,:r2] @ np.diag(s[:r2]) # r1*n3 x r2
    U2 = U2.reshape(r1, n3, r2) # r1 x n3 x r2
    U2 = U2.transpose([0,2,1]) # r1 x r2 x n3
    U2 = U2.reshape(r1 * r2, n3) # r1*r2 x n3
    U, s, V = np.linalg.svd(U2, full_matrices=False)
    V3 = V[:r3,:] # U3(r3,j)
    G = U[:,:r3] @ np.diag(s[:r3])# r1 * r2 x r3
    G = G.reshape(r1, r2, r3) # main core
    return G, V1, V2, V3

def Check(A, G, V1, V2, V3): 
    err = 0
    r1, r2, r3 = G.shape
    A1 = G.reshape(r1 * r2, r3).dot(V3) # r1*r2 x n3
    A1 = A1.reshape(r1, r2, V3.shape[1]).transpose([0,2,1]) # r1 x r2 x n3
    A1 = A1.reshape(r1*V3.shape[1], r2).dot(V2) # r1*n3 x n2
    A1 = A1.reshape(r1, V3.shape[1], V2.shape[1]).transpose([1,2,0]) # n3 x n2 x r1
    A1 = A1.reshape(V3.shape[1], V2.shape[1], r1).dot(V1) # n3 x n2 x n1
    A1 = A1.transpose([2,1,0]) # n1 x n2 x n3
    return np.linalg.norm(A - A1) #error norm

In [261]:
%%time

N = 64
A = np.zeros([N,N,N])
for i in range(N):
    for j in range(N):
        for k in range(N):
            A[i,j,k] = np.sin(i+2.*j+3.*k)
G, V1, V2, V3 = HO_SVD_3d(A,4,4,4)
print("Error =", Check(A, G, V1,V2,V3))

Error = 1.9423889953547366e-12
CPU times: user 350 ms, sys: 9.52 ms, total: 359 ms
Wall time: 313 ms


### st_HO_SVD

In [262]:
def st_HO_SVD(A):
    N = A.shape[0]
    U_1, E, V = np.linalg.svd(np.reshape(A, (N, N * N)), full_matrices=False)
    D = np.diag(E) @ V
    D = np.transpose(D.reshape([N, N, N]), [1, 0, 2]).reshape([N, N*N])
    U_2, E, V = np.linalg.svd(D, full_matrices=False)
    D = np.diag(E) @ V
    D = np.transpose(D.reshape([N, N, N]), [2, 1, 0]).reshape([N, N*N])
    U_3, E, V = np.linalg.svd(D, full_matrices=False)
    D = (np.diag(E) @ V).reshape([N, N, N])
    return D, U_1, U_2, U_3

In [263]:
A = np.array([[[1, 2], [3, 4]], [[5, 6], [7, 8]]])
st_HO_SVD(A)

(array([[[-1.42253953e+01, -8.28025332e-03],
         [-4.61793060e-03,  1.11585148e+00]],
 
        [[-1.60125603e-02,  2.38589095e-01],
         [ 5.43770692e-01, -2.00114739e-01]]]),
 array([[-0.37616823,  0.92655138],
        [-0.92655138, -0.37616823]]),
 array([[ 0.56672424, -0.82390754],
        [ 0.82390754,  0.56672424]]),
 array([[ 0.64142303, -0.7671874 ],
        [ 0.7671874 ,  0.64142303]]))

# TT_SVD

In [264]:
def TT_SVD(A, ranks):
    g = []
    n = list(A.shape)
    dim = len(n)

    A1 = A.reshape(n[0], np.prod(n[1:]))
    U, s, V = np.linalg.svd(A1, full_matrices=False)
    G1 = U[:,:ranks[0]]
    g.append(G1)
    B = np.diag(s[:ranks[0]]) @ V[:ranks[0], :]
    for k in range (1, dim - 1):
        B = B.reshape(ranks[k - 1] * n[k], np.prod(n[k+1:]))
        U, s, V = np.linalg.svd(B, full_matrices=False)
        Gk = U[:,:ranks[k]]
        Gk = Gk.reshape(ranks[k - 1], n[k], ranks[k])
        g.append(Gk)
        B = np.diag(s[:ranks[k]]) @ V[:ranks[k],:]
    g.append(B)
    return g

def Restore_tensor(g):
    n = [g[i].shape[1] for i in range(len(g))]
    n[0] = g[0].shape[0]
    dim = len(n)
    
    res = g[-1]
    for i in range(dim - 2, 0, -1):
        res = (g[i] @ res).reshape(g[i].shape[0], np.prod(n[i:dim]))    
    res = g[0] @ res
    res = res.reshape(n)
    return res

In [265]:
%%time

N = 32
A = np.zeros([N,N,N])
for i in range(N):
    for j in range(N):
        for k in range(N):
            A[i,j,k] = np.sin(i+2.*j+3.*k)
            
g = TT_SVD(A, [4, 4])
B = Restore_tensor(g)

err = np.linalg.norm(A - B)
print("error = ", err)

error =  8.901142497836701e-13
CPU times: user 67.4 ms, sys: 5.44 ms, total: 72.8 ms
Wall time: 62 ms


# ALS

In [266]:
def ALS(A, T, dimensions, rank):
    d = len(dimensions)
    for i in range(10000):
        for j in range(0, d):
            B = np.full((1, rank), fill_value=1)
            C = np.full((rank, rank), fill_value=1)
            for k in range(0, d):
                if (k != j):
                    C = np.multiply(C, A[k].T @ A[k])
                    B = linalg.khatri_rao(B, A[k])
            Y = B.T @ np.moveaxis(T, j, 0).reshape(T.shape[j], -1).T
            A[j] = (linalg.pinv(C) @ Y).T
    res = np.einsum('ir,jr,kr,wr', *A)
    err = np.linalg.norm(res - T) / np.linalg.norm(T)
    print(err)
    return A, err

In [267]:
N = 10
dim = (N, N, N, N)
rank = (1, 2, 4, 8, 16)
tensor = np.zeros([N,N,N,N])
for i in range(N):
    for j in range(N):
        for k in range(N):
            for q in range(N):
                tensor[i,j,k, q] = np.sin(i+2*j+3*k) + 4*q
for j in rank:
    A = [np.random.normal(size=(i, j)) for i in dim]
    A, err = ALS(A, tensor, dim, j)

0.03315319015555635
0.0236105164344558
2.1584386748628728e-14
0.00010190671076600212
6.069735406989148e-06
